In [ ]:
#########################################################
####
#### Tutorial: Alignment with Infrared (for Developers)
####
#########################################################

import time

import infrared as ir
from infrared import Model, BoltzmannSampler, def_function_class, def_constraint_class, rna
import math
import treedecomp

In [ ]:
# start by defining classes for representing and scoring alignments

def is_compl(x,y):
    return (x+y) in ["AU","CG","GC","GU","UA", "UG"]

class Alignment:
    def __init__( self, assignment, a, s, b ):
        self._a = a
        self._s = s
        self._b = b
        self._assignment = assignment
        self._values = assignment.values()
        self._edges = self._values_to_edges(self.values)
    
    @property
    def values(self):
        return self._values

    @property
    def edges(self):
        return self._edges

    @staticmethod 
    def _values_to_edges(values):
        edges = list()
        for x,x1 in zip(values,values[1:]):
            edges.append(x1-1 if x<x1 else -1)
        edges = edges[:-1]
        return (edges)
    
    def _alignment_strings(self):
        s = list()
        b = list()
        a = list()
        t = list()
         
        for i,(x,x1) in enumerate(zip(self._values,self._values[1:])):
            if x<x1:
                for j in range(x,x1-1):
                    a.append('-')
                    s.append('-')
                    b.append(self._b[j])
                if i<len(self._a):
                    a.append(self._a[i])
                    s.append(self._s[i])
                    b.append(self._b[x1-1])
            if x==x1:
                if i<len(self._a):
                    a.append(self._a[i])
                    s.append(self._s[i])
                    b.append('-')

        t = [ '-' if x=='-' else '.' for x in b ]
        s = "".join(s)
        for (i,j) in rna.parseRNAStructureBps(s):
            if is_compl(b[i],b[j]):
                t[i]=s[i]
                t[j]=s[j]
                    
        return [ s, "".join(a), "".join(b), "".join(t)]
    
    def __str__(self):
        alignment = self._alignment_strings()
        return "\n".join(alignment)

    
class AliScore():
    def __init__(self, a, b):
        self._a = a
        self._b = b
    def beta( self ):
        return -1
    def gamma( self, k = 1 ):
        return -2 + self.beta()*k
    def sigma( self, x, y ):
        return 4 if self._a[x-1]==self._b[y-1] else 0
    def psi( self, xi, xj ):
        return 8 if is_compl(self._b[xi-1],self._b[xj-1]) else 0

In [ ]:
## first, define the problem-specific types/classes of constraints and functions

## constraints and functions use var to translate named variables to internal indices

def_constraint_class( 'LeqConstraint', lambda i,var: var([('X',i-1),('X',i)]),
                      lambda x1,x: x1<=x )

def_constraint_class( 'EqConstraint', lambda thevar,c,var: var([thevar]), lambda x,c: x==c )

def_constraint_class( 'XYRelation', lambda i,var: var([('X',i-1),('X',i),('Y',i)]),
                      lambda x1,x,y: (x1<x and y==1) or (x1==x and y==0) )

def_constraint_class( 'BandingConstraint', lambda i,c,n,m,var: var([('X',i)]),
                      lambda x,i,c,n,m: abs(x-i*m/n) <= c )


## no weights in the functions definitions
def_function_class( 'SigmaFunction', lambda i,score,var: var([ ('X',i), ('Y',i) ]),
                     lambda x,y,i,score: score.sigma(i,x) if y==1 else 0 )

def_function_class( 'InsertFunction', lambda i,score,var: var([('X',i-1),('X',i)]),
                    lambda x1,x,score: score.gamma(x-x1) )

def_function_class( 'DeleteFunction', lambda i,score,var: var([('Y',i-1),('Y',i)]),
                    lambda y1,y,score: ( score.beta() if y1==0 else score.gamma(1) ) if y==0 else 0 )

def_function_class( 'PhiFunction', lambda i,j,score,var: var([('X',i),('Y',i),('X',j),('Y',j)]),
                    lambda xi,yi,xj,yj,score: score.psi(xi,xj) if yi==1 and yj==1 else 0 )



In [ ]:
## use constraints and functions to define the alignment model for a specific instance

#DOMAIN_BANDING = False
DOMAIN_BANDING = True

## a small test instance
a = "ACUUG"
s = "([.)]"
b = "AGGAUC"

## simple crossing
a = "CGCCAAUACAAUAGGGUUUAU"
s = "(.((.[[[.....)))..]]]"
b = "GCGCAAACAAGCGAAUUUUAA"

## Some interesting hand-crafted example
a = "CGCCAAUAAUAGGGUUUAU"
s = "(.(([[.[{{.)))]]]}}"
b = "GCGCAAACAAGCGAAUUUUU"
#b += b

## Licorna example RF01099 ( Treewidth 3 / 7 )
#a = "UUCCAGGACAUACUGCUGAGGAUGUCAAAAAUGCAGUUGGAGUCCUCA"
#s = ".<<<<<...........((((((.............>>>>>))))))."
#b = "UUCCAGGACAUACUGAUGAGAAUGUCAAAGAUGCAGUUGGGGUCCUCA"

## Licorna example RF01831 ( Treewidth 4 / 9 )
#a = "UCAGAGUAGAAAACGAUGCGUUAAGUGUCCAGCAGACGGGGAGUUGCUGCCGGAACGAAAAGCAAAGCUUGCGGUAUCGUUUUCGCAUCCCGCUGA"
#s = "<<<<....<<<<<<<<<<<.....<<.<<<.<<<<...(((.((..>>>>.>>>>>...<<<<...>>>>...>>>>>>>>>>>)).)))..>>>>"
#b = "GCAGAGUAGACACAUGUGCGUUAAGUGCCGGAUGAACAGGGAGUUGUCACCCGGACGAAAAGAAAAUCUUGCGGUACAUGAGUCGCAUCCCGCUGC"


## Licorna example RF01786 (challenging b/c this enforces large c)
#3Q3Z.seq vs target 2
a = "CCCCGGAAACAAAAACGAAUGUGAAUGGGUUGGGCAUGGGCACUAGACUCAUUUUGUGUUGCAAGUGCACCCGA"
s = "............((((.....((((((((((.....<[[[[[[[.))))))))))..)))...]]]]]>.]].)"
b = "ACUGCAAUGGGUGUGAUGAAGUCCGGACAGUAAUGUGGGCACUUAGUCCGGACCGAGCAAGUAGUGCAACCGACCAGAUGCAAA"

c=5
c = max(c,abs(len(a)-len(b)))

n = len(a)
m = len(b)

print(s)
print(a,n)
print(b,m)

bps = rna.parseRNAStructureBps(s)
score = AliScore( a, b )

model = Model()

# X_i's encode alignments, such that 
#  * X_i=j for alignment edges (i,j) (1-based pos indices!) and
#  * X_i==X_i-1 if X_i is deleted
#  * X_0 = 0
#  * X_n+1=m+1
model.add_variables( 1, (0,0), name = 'X' )
model.add_variables( n, m+1, name = 'X' )
model.add_variables( 1, (m+1,m+1), name = 'X' )

# Y_i is 1 iff i is matched
model.add_variables( n+1, 2, name = 'Y' )

var = lambda vs: model.idx(vs)

model.add_constraints( LeqConstraint( i, var ) for i in range(1,n+2) )
model.add_constraints( XYRelation( i, var ) for i in range(1,n+1) )

#model.add_constraints( EqConstraint( ('X',n+1), m+1, var ) )
#model.add_constraints( EqConstraint( ('Y',0), 1, var ) )

model.restrict_domains( ('Y',1), (1,1) )


if DOMAIN_BANDING:
    for i in range(1,n+1):
        lb = max(1, math.floor(i*m/n) - c)
        ub = min(n, math.ceil(i*m/n) + c )
        model.restrict_domains( ('X',i), (lb,ub) )    
else:
    model.add_constraints( BandingConstraint( i, c, n, m, var ) for i in range(1,n+1) )

model.add_functions( [ SigmaFunction( i, score, var )
                       for i in range( 1, n+1 ) ], group = 'sigma' )
model.add_functions( [ InsertFunction( i, score, var )
                       for i in range( 1, n+2 ) ], group = 'indels' )
model.add_functions( [ DeleteFunction( i, score, var )
                       for i in range( 1, n+1 ) ], group = 'indels')

model.add_functions( [ PhiFunction(i+1, j+1, score, var) for (i,j) in bps ], group = 'phi' )

#### not yet available, potential future features/syntax
## optimizer for the model
#alioptimizer = ArcticOptimizer( model )
#
#solution = alioptimizer.optimum()
#score = model.eval_feature( solution )
#print("Score", score )
#print( Alignment( solution, a, b ) )
#
# get suboptimal solutions from optimizer 
#suboptimals = alioptimizer.suboptimals( threshold = score - 2 )

# treedecomp factory with light internal optimization
td_factory = treedecomp.NXTreeDecompositionFactory( iterations=1, adaptive=None )

# sampler for the model
alisampler = BoltzmannSampler( model, td_factory = td_factory )

## set weights
w = 2
model.set_feature_weight(w,"sigma")
model.set_feature_weight(w,"indels")
model.set_feature_weight(w,"phi")

#### not yet available, potential future feature/syntax
# tell sampler to produce non-redundant samples
# alisampler.set_non_redundant( True )

t0 = time.time()


## optimize tree decomposition --- this optimization should be moved to TD Factory entirely
best_domsizeprod = None

i=0
while True:
    i+=1
    tw = alisampler.treewidth()
    
    td = alisampler._td
    alisampler._td = None
    
    bags = td.get_bags()
    
    import math
    domsizeprods = [ math.prod( model.all_domains[x].size() for x in bag ) for bag in bags ]
    domsizeprod = sum(domsizeprods)
    
    if best_domsizeprod==None or domsizeprod < best_domsizeprod:
        best_domsizeprod = domsizeprod
        best_td = td

        ## some magic formula to try harder for harder instances, not super-optimized
        thresh = 33 * 1.5**(domsizeprod/3e8)
        #print('Thresh',thresh)
    if i > thresh:
        break

print( "After iteration",i,"Treewidth:",tw,end=", ")
print( "Effective Treewidth:", max(len( [ x for x in b if x <= len(a)+1 ]) for b in bags) -1, end=", " )
print( f"Domsize product sum: {best_domsizeprod:e}" )

        
alisampler._td = best_td
    
print("Banding:", c)

alisampler.plot_td("treedecomp.pdf")
      
print(f"t = {time.time()-t0}s")

alisampler.setup_engine()
pf = alisampler.ct.evaluate()

print("Partition function",pf)
print(f"t = {time.time()-t0}s")


## and print samples
for _ in range(10):
    sample = alisampler.sample()
    print()
    sigma = model.eval_feature( sample, 'sigma' )
    indels = model.eval_feature( sample, 'indels' )
    phi = model.eval_feature( sample, 'phi' )
    print(f"Score {sigma + indels + phi} = {sigma} + {indels} + {phi}" )
    # print(sample.values())
    print( Alignment( sample, a, s, b ) )
    
print()
print(f"t = {time.time()-t0}s")

import os, psutil
process = psutil.Process(os.getpid())
print(f'RSS {process.memory_info().rss/1024:.1f} kb')  # in bytes 
